### **MAKE A COPY OF THIS DOCUMENT BEFORE START WORKING ON IT**


Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Upload offline conversions to Campaign Manager



> **What is?**


CM Offline Conversions uploader is sample code that can easily be run by advertisers. With the CM Offline Conversions uploader, you can easily import conversions and build audiences out of those conversions. This is a great solution if you are looking for:

*  Uploading conversions that have been impacted by a campaign of the advertiser

*   Optimize a campaign based on offline Conversions

*   Build Audiences out of the conversions that you can upload.



> **What do I need to use this Datapill?**

1.   Make a copy of this Colab (It is ready to go!)
2.   Create a spreadsheet in the Google Drive of the person that will run the cells of this DataPill (colab). The spreadsheet must have the information of the gclids or Device_IDs listed in the first column. There should be 1 ID per row. 
3.   Create Sercvice Account and give it access to the Campaign Manager Advertiser

For more detail visit [this document](https://docs.google.com/document/d/1P9ApL9_lMna53LYB6LzgHYkq_ecJaI5imT2FDG83fmg/edit?usp=sharing)




> **Is there anything else I need to know?**


1.   To upload Offline conversions based on Device_IDs (IDFA,ADID) they should have been exposed to a Campaign Manager's Campaign from the Advertiser who is trying to upload the conversions.




> **Documentation**

[Conversions batchUpdate via API](https://developers.google.com/doubleclick-advertisers/v3.3/conversions/batchupdate) 

[FAQ Campaign Manager API](https://developers.google.com/doubleclick-advertisers/guides/conversions_faq)



> **Disclaimers**

Copyright 2019 Google LLC. This solution, including any related sample code or data, is made available on an “as is,” “as available,” and “with all faults” basis, solely for illustrative purposes, and without warranty or representation of any kind. This solution is experimental, unsupported and provided solely for your convenience. Your use of it is subject to your agreements with Google, as applicable, and may constitute a beta feature as defined under those agreements.  To the extent that you make any data available to Google in connection with your use of the solution, you represent and warrant that you have all necessary and appropriate rights, consents and permissions to permit Google to use and process that data.  By using any portion of this solution, you acknowledge, assume and accept all risks, known and unknown, associated with its usage, including with respect to your deployment of any portion of this solution in your systems, or usage in connection with your business, if at all.




---



---



# **Upload Necesary information for Offline conversions**

**key_type** needs to have one of those two values:  
- gclid
- mobileDeviceId


In [ ]:
# Model parameters
floodlightActivityId =  123123#@param {type:"integer"}
floodlightConfigurationId = 3123123 #@param {type:"integer"}
credentials_dict = "" #@param {type:"string"}
profileId_CM = "1231231" #@param {type:"string"}
sheet_name = "devices_ids" #@param {type:"string"}
key_type = "mobileDeviceId" #@param {type: "string"}


In [ ]:
print(credentials_dict)

In [ ]:
!pip install --upgrade -q gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())



In [ ]:
import json
import httplib2
import time
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient import discovery


def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]


def get_conversions():
    conversions = []
    conversion = {
        'floodlightActivityId': floodlightActivityId,
        'floodlightConfigurationId': floodlightConfigurationId,
    }  
    worksheet = gc.open(sheet_name).sheet1

    # get_all_values gives a list of rows.
    results = worksheet.get_all_values()

    customVars = []

    for row in results:
          
      current_time_in_micros = int(time.time() * 1000000)
      conversion[key_type] = row[0]
      conversion['ordinal'] = current_time_in_micros
      conversion['timestampMicros'] = current_time_in_micros
      # customVars.append(
      #   {
      #     "kind": "dfareporting#customFloodlightVariable",
      #     "type": 'u1',
      #     "value": row[1]
      #   }
      # )
      

      # conversion['customVariables'] = customVars
      print(conversion)
      conversions.append(conversion)
    return conversions


def send_conversions(conversion, service):
    request_body = {
      'conversions': conversion,
    }
    request = service.conversions().batchinsert(profileId= profileId_CM,
                                            body=request_body)
    
    ### SHOULD WE CHANGE THIS PROFILE ID 
    
    response = request.execute()

    print(response)




def get_service_auth():
  OAUTH_SCOPES = ['https://www.googleapis.com/auth/dfareporting', 'https://www.googleapis.com/auth/ddmconversions', 'https://www.googleapis.com/auth/dfatrafficking']



  json_data = json.loads(credentials_dict, strict=False)


  """Authorizes an httplib2.Http instance using service account credentials."""
  # Load the service account credentials from the specified JSON keyfile.
  credentials = ServiceAccountCredentials.from_json_keyfile_dict(json_data, scopes=OAUTH_SCOPES)
 
  # Use the credentials to authorize an httplib2.Http instance.
  http = credentials.authorize(httplib2.Http())

  # Construct a service object via the discovery service.
  service = discovery.build('dfareporting', 'v3.3', http=http)
  
  return service


def main():
    data = get_conversions()
    print("________________________________________")    
    print("Sendind chunks")
    service = get_service_auth()
    conversions = chunks(data, 100)
    for conversion in conversions:
      print(conversion)
      send_conversions(conversion, service)
    return "Done"

main()

